# Import

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
# import dask
import dask.dataframe as dd
import geopandas as gpd

from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp
from datetime import date

from tqdm.notebook import tqdm
from dask.diagnostics import ProgressBar

pd.options.mode.chained_assignment = None  # default='warn'

# Data

In [3]:
%%time
# Folders
repository = Path.cwd()
dataFolder = repository.parent.parent / 'InOutRepoData' / 'FFE'
folder = repository / 'data' / 'comparison_arcpy'

CPU times: user 218 µs, sys: 192 µs, total: 410 µs
Wall time: 259 µs


In [4]:
%%time
# Data

wind_data = pd.read_csv(repository / 'data' / 'Copy_of_GD_wind.csv')
edgelist = pd.read_parquet(repository / 'data' / 'BLDG_ID_ShapeEdges.parquet', engine='pyarrow')

CPU times: user 269 ms, sys: 269 ms, total: 537 ms
Wall time: 348 ms


**data from the scenario HikWgtnMin_pfour_1-100     
scenario1 initial ignition BLDG_ID : 217, 686, 32, 166, 15, 937, 46 which translate into edge_id 216, 685, 31, 165, 14, 936, 45**

**Wind speed - critical distance 45m
Wind direction NW**

**18min processing time**

In [5]:
rngdata = {'source':[6041, 19624, 21701, 28758, 
                     40702,41549, 58068], 
           'IgnProb_bl':[1, 1, 1, 1, 1, 1, 1]} 
rngFile = pd.DataFrame(rngdata)

## Parallel computing set up

## Functions

In [19]:
# %%px

def wind_scenario(wind_data):
    import numpy as np
    i=6407 # pick particular wind event (45m distance, direction NW)
#     i = np.random.randint(0, wind_data.values.shape[0])
    w = wind_data.values[i, 2]
    dist = wind_data.values[i, 1]
    b = wind_data.values[i, 3]
    bear_max = b + 45  # wind direction
    bear_min = b - 45
    if b == 360:
        bear_max = 45
    if b <= 0:  # should not be necessary
        bear_min = 0
    if b == 999:
        bear_max = 999
        bear_min = 0
    print(f"w_direction : {b}, w_bearing_max : {bear_max}, w_bearing_min : {bear_min}, w_distance : {dist}")
    return bear_max, bear_min, dist  # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    import numpy as np
    import pandas as pd
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProb_bl']]
    initialIgnitions = len(rngList)
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges, initialIgnitions


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t == 0:  # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (
            activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(
            listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges

In [20]:
# @dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    from datetime import date
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges, numberIgnitions = ignition()
        if ActiveEdges.empty:
            print(f"no ignitions {numberIgnitions}")
            condition = False
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            burns = ActiveEdges.drop_duplicates(['source'], inplace=False)
#             print(f"Active edges {len(ActiveEdges)} / no duplicate = {len(burns)}")
            burns['time'] = time
            listScenarioDataframes.append(burns)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing scenario --- {scenario} time ---- {time} \n started with {numberIgnitions} ignitions ')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["InitialIgnitions"] = numberIgnitions
        Activations.to_parquet(folder / f'wind9807_scenario{scenario}_{str(date.today())}.parquet', engine='auto', compression="GZIP")

# Run

In [ ]:
%%time
ffe_runs(range(1))

w_direction : 315, w_bearing_max : 360, w_bearing_min : 270, w_distance : 45



### Post processing and comparison

In [ ]:
repoFolder = Path.cwd()
comparisonFolder = repoFolder.parent.parent / 'InOutRepoData' / 'FFE' / 'ProbaScenariosInput' / 'comparisonArcPyResults'

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt

In [ ]:
%%time
testNetScenario = pd.read_parquet(folder / f'wind9807_scenario0_{str(date.today())}.parquet')
testArc = gpd.read_file(comparisonFolder / 'HikWgtnMin_pfourARCPY.shp')
testArcScenario = testArc.iloc[:,:9]
del testArc

In [ ]:
%%time
# merge geometry to scenario
# merge each line / source with geometry 
shape = gpd.read_file('data/shapefile/BuildingFootprints.shp')
shape = shape[['BLDG_ID', 'geometry']]
# shape
ScenarioARC = pd.merge(testArcScenario, shape, how='left', on='BLDG_ID')
ScenarioNET = pd.merge(testNetScenario, shape, how='left', left_on='source', right_on='BLDG_ID')
# create as Geodataframe
ScenarioARC = gpd.GeoDataFrame(ScenarioARC, geometry=ScenarioARC.geometry)
ScenarioNET = gpd.GeoDataFrame(ScenarioNET, geometry=ScenarioNET.geometry)
ScenarioNET = ScenarioNET.set_crs(epsg=2193)

# save as shapefile
ScenarioARC.to_file(comparisonFolder / 'ScenarioARC.shp',driver ='ESRI Shapefile') #, driver="GeoJSON")
ScenarioNET.to_file(comparisonFolder / 'wind9807__ScenarioNET.shp',driver ='ESRI Shapefile') #, driver="GeoJSON")

#### Create Gif for comparisons 

In [ ]:
ScenarioARC.Ignited1.max()
ScenarioNET.time.max()

In [ ]:
def save_ignitionsMaps(geoARC, geoNET, columnNameARC, columnNameNET):
    mapShape = gpd.read_file('data/shapefile/BuildingFootprints.shp')
    for i in tqdm(range(max(geoNET[columnNameNET]))):
        ImapARC = geoARC[(geoARC[columnNameARC]<=i) & (geoARC[columnNameARC]>0)]
        ImapNET = geoNET[geoNET[columnNameNET]<=i]

        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(15, 10))

        ImapARC.plot(color="red", edgecolor=None, ax=ax1)
        mapShape.geometry.boundary.plot(color=None,edgecolor='k',linewidth = 0.1,ax=ax1)
        ImapNET.plot(color="red", edgecolor=None, ax=ax2)
        mapShape.geometry.boundary.plot(color=None,edgecolor='k',linewidth = 0.1,ax=ax2)

        ax1.set_title('ArcPy scenario')
        ax1.ticklabel_format(useOffset=False, style='plain')
        ax1.tick_params(direction='out', length=6)
        ax2.set_title('Network scenario')
        ax2.ticklabel_format(useOffset=False, style='plain')
        ax2.tick_params(direction="out", length=6)

        fig.autofmt_xdate()
        plt.tight_layout()
        plt.savefig(comparisonFolder / 'comparisonGIF' / f"{i}.png", dpi=70)
        plt.close()

In [ ]:
save_ignitionsMaps(ScenarioARC, ScenarioNET, "Ignited1", "time")